In [ ]:
# Select the TensorFlow 2.0 runtime
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
# Install Weights and Biases (WnB)
!pip install wandb

In [ ]:
# Primary imports
import tensorflow as tf
import numpy as np
# import wandb

In [ ]:
# Authorize Weights and Biases
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: ab6c21fa98b79790d8c7a9d0a0844b7a1ddaa4e2
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [ ]:
# Intialize WnB with a project name of your choice
wandb.init(project="custom_training_loops_tf-hist")

W&B Run: https://app.wandb.ai/sayakpaul/custom_training_loops_tf-hist/runs/288lxmyz

In [ ]:
# Load the FashionMNIST dataset, scale the pixel values
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.

X_train.shape, X_test.shape, y_train.shape, y_test.shape

4423680/4422102 [==============================] - 0s 0us/step


((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [ ]:
# Define the labels of the dataset
CLASSES=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [ ]:
# Change the pixel values to float32 and reshape input data
X_train = X_train.astype("float32").reshape(-1, 28, 28, 1)
X_test = X_test.astype("float32").reshape(-1, 28, 28, 1)

In [ ]:
y_train.shape, y_test.shape

((60000,), (10000,))

In [ ]:
# TensorFlow imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [ ]:
# Define utility function for building a basic shallow Convnet 
def get_training_model():
    model = Sequential()
    model.add(Conv2D(16, (5, 5), activation="relu",
        input_shape=(28, 28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(len(CLASSES), activation="softmax"))
    
    return model

In [ ]:
# Define loass function and optimizer
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
# Average the loss across the batch size within an epoch
train_loss = tf.keras.metrics.Mean(name="train_loss")
valid_loss = tf.keras.metrics.Mean(name="test_loss")

# Specify the performance metric
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="train_acc")
valid_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="valid_acc")

In [ ]:
# Batches of 64
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)

In [ ]:
# Train the model
@tf.function
def model_train(features, labels):
    # Define the GradientTape context
    with tf.GradientTape() as tape:
        # Get the probabilities
        predictions = model(features)
        # Calculate the loss
        loss = loss_func(labels, predictions)
    # Get the gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    # Update the weights
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Update the loss and accuracy
    train_loss(loss)
    train_acc(labels, predictions)

In [ ]:
# Validating the model
@tf.function
def model_validate(features, labels):
    predictions = model(features)
    v_loss = loss_func(labels, predictions)

    valid_loss(v_loss)
    valid_acc(labels, predictions)

In [ ]:
# A shallow Convnet
model = get_training_model()

In [ ]:
# Grab random images from the test and make predictions using 
# the model *while it is training* and log them using WnB
def get_sample_predictions():
    predictions = []
    images = []
    random_indices = np.random.choice(X_test.shape[0], 25)
    for index in random_indices:
        image = X_test[index].reshape(1, 28, 28, 1)
        prediction = np.argmax(model(image).numpy(), axis=1)
        prediction = CLASSES[int(prediction)]
        
        images.append(image)
        predictions.append(prediction)
    
    wandb.log({"predictions": [wandb.Image(image, caption=prediction) 
                               for (image, prediction) in zip(images, predictions)]})

In [ ]:
# Train the model for 5 epochs
for epoch in range(5):
    # Run the model through train and test sets respectively
    for (features, labels) in train_ds:
        model_train(features, labels)

    for test_features, test_labels in test_ds:
        model_validate(test_features, test_labels)
        
    # Grab the results
    (loss, acc) = train_loss.result(), train_acc.result()
    (val_loss, val_acc) = valid_loss.result(), valid_acc.result()
    
    # Clear the current state of the metrics
    train_loss.reset_states(), train_acc.reset_states()
    valid_loss.reset_states(), valid_acc.reset_states()
    
    # Local logging
    template = "Epoch {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}"
    print (template.format(epoch+1,
                         loss,
                         acc,
                         val_loss,
                         val_acc))
    
    # Logging with WnB
    wandb.log({"train_loss": loss.numpy(),
               "train_accuracy": acc.numpy(),
               "val_loss": val_loss.numpy(),
               "val_accuracy": val_acc.numpy()
    })
    get_sample_predictions()

Epoch 1, loss: 0.547, acc: 0.802, val_loss: 0.401, val_acc: 0.854
Epoch 2, loss: 0.350, acc: 0.873, val_loss: 0.356, val_acc: 0.872
Epoch 3, loss: 0.300, acc: 0.892, val_loss: 0.334, val_acc: 0.878
Epoch 4, loss: 0.271, acc: 0.901, val_loss: 0.322, val_acc: 0.880
Epoch 5, loss: 0.247, acc: 0.909, val_loss: 0.301, val_acc: 0.888


In [ ]:
model.save("fashion-mnist-kf-sm")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: fashion-mnist-kf-sm/assets


In [ ]:
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        print(layer.get_weights()[0].shape)
        wandb.log({"weights": wandb.Histogram(layer.get_weights()[0])})
        wandb.run.summary.update({"weights": wandb.Histogram(layer.get_weights()[0])})

(5, 5, 1, 16)
(5, 5, 16, 32)
(512, 128)
(128, 10)


In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=0CmX9NGkR4VskNSO0pm1QkcpJTNvKnVCdi64LOrxda0&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/3gHOmbzJ2DSx4Jw42L7eSpOqK38aCsoOCeWOd-PMaYYRNFH1gG_mjQ0

You are now logged in as [spsayakpaul@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
!gcloud config set project fast-ai-exploration

Updated property [core/project].


In [ ]:
!gsutil cp -r fashion-mnist-kf-sm/* gs://fashion-mnist-kf-sm/

Copying file://fashion-mnist-kf-sm/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://fashion-mnist-kf-sm/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://fashion-mnist-kf-sm/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
\
Operation completed over 3 objects/425.4 KiB.                                    


In [ ]:
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        print(layer.trainable_variables[0].shape)

(5, 5, 1, 16)
(5, 5, 16, 32)
(512, 128)
(128, 10)
